In [55]:
# Set up the client
from openai import OpenAI
from dotenv import load_dotenv
import os
import requests
import time

load_dotenv()  # take environment variables from .env.

assistant_id = os.getenv("ASSISTANT_ID")
server_url = os.getenv("SERVER_URL")
client = OpenAI(api_key=os.getenv("OPEN_AI_KEY"))


In [56]:
instructions=[
    "Hello plz tell how many repo i have on github. Id: ethansnow2012",
    "Who has most followers on github?",
]

def get_github_profile(id=''):
    #print("Getting github profile for user with id: ", id)
    response = requests.get(server_url)
    return response.text

get_github_profile()

'[{"id": "ethansnow2012", "content": "Skip to content\\nethansnow2012\\nOverview\\nRepositories\\n54\\nProjects\\nPackages\\nStars\\n48\\nSet status\\nethansnow2012\\nEdit profile\\n 3 followers \\u00b7 10 following\\nAchievements\\nx3\\nBetaSend feedback\\nOrganizations\\nethansnow2012/README.md\\nHi there \\ud83d\\udc4b\\n\\ud83d\\udd2d I\\u2019m currently working on (Frontend Development|Product Strategy).\\n\\ud83d\\ude38 I\\u2019m tying to build a SaaS as an experiment for myself. Beta Production: Live Demo Demo: Figma Business Discussion: Figjam\\n\\ud83d\\udd2d I\\u2019m looking for job opportunities. About me: Cakeresume\\n\\n\\nRecently working on:\\nCodeBase Export Tool for Ai: CodeBaseJson\\nSaaS Platform for Larps: \\u7dad\\u514b\\u591a\\u9152\\u5427\\n\\n\\nMy Gist\\n\\nhttps://gist.github.com/ethansnow2012\\n\\nCustomize your pins\\nPinned\\n CodeBaseJson Public\\n\\nParse your code base to Json so that Ai can read it.\\n\\n TypeScript  1\\n\\nContribution settings \\n751

In [57]:
# Fetch the assistant

assistant = client.beta.assistants.retrieve(
    assistant_id=assistant_id
)
assistant

Assistant(id='asst_pc2Su2UHd6IV7i3WwaYAhCqF', created_at=1712382025, description=None, file_ids=[], instructions='Helping me to find the write id(person is nice).', metadata={}, model='gpt-4', name='github profile', object='assistant', tools=[FunctionTool(function=FunctionDefinition(name='get_github_data', description='Retrieves data from the API using id as filter if needed. This function is called only if the data has not been previously retrieved or need to be retrived again.', parameters={'type': 'object', 'properties': {'github_id': {'type': 'string', 'description': 'The GitHub ID for which the data needs to be fetched'}}, 'optional': ['github_id'], 'required': []}), type='function')])

In [58]:
# Create a thread

thread = client.beta.threads.create()
thread

Thread(id='thread_o8t2nKJzhk0bKprh0YYd4Rpv', created_at=1712411505, metadata={}, object='thread')

In [59]:
# Prompt the model to tell us all about the data provided

run = client.beta.threads.runs.create(
    thread_id = thread.id,
    assistant_id = assistant.id,
    instructions = instructions[1]
)
run

Run(id='run_qvTZL2sKjduAkz63iXiBar9v', assistant_id='asst_pc2Su2UHd6IV7i3WwaYAhCqF', cancelled_at=None, completed_at=None, created_at=1712411506, expires_at=1712412106, failed_at=None, file_ids=[], instructions='Who has most followers on github?', last_error=None, metadata={}, model='gpt-4', object='thread.run', required_action=None, started_at=None, status='queued', thread_id='thread_o8t2nKJzhk0bKprh0YYd4Rpv', tools=[FunctionTool(function=FunctionDefinition(name='get_github_data', description='Retrieves data from the API using id as filter if needed. This function is called only if the data has not been previously retrieved or need to be retrived again.', parameters={'type': 'object', 'properties': {'github_id': {'type': 'string', 'description': 'The GitHub ID for which the data needs to be fetched'}}, 'optional': ['github_id'], 'required': []}), type='function')], usage=None, temperature=1.0)

In [60]:


# Function to retrieve the run
def retrieve_run(thread_id, run_id):
    run = client.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run_id)
    return run

# Function to process tool calls
def process_tool_calls(run):
    tool_calls = run.required_action.submit_tool_outputs.tool_calls
    tool_outputs = [get_outputs_for_tool_call(tool_call) for tool_call in tool_calls]
    return tool_outputs

# Initial setup
thread_id = thread.id
run_id = run.id
run = retrieve_run(thread_id, run_id)

# Attempt counter
attempts = 0

time.sleep(2.5)
# Loop until the required action is true or attempts limit reached
while not run.required_action and attempts < 3:
    print(f"Attempt {attempts + 1}: Waiting for job to complete...")
    time.sleep(5)  # Wait for 10 seconds before trying again
    run = retrieve_run(thread_id, run_id)
    attempts += 1

# Check if required action is true
if run.required_action:
    print("Job completed, processing tool calls...")
    tool_outputs = process_tool_calls(run)
    # Submit tool outputs
    run = client.beta.threads.runs.submit_tool_outputs(
        thread_id=thread_id,
        run_id=run_id,
        tool_outputs=tool_outputs
    )
    print("Process completed.")
else:
    print("Job did not complete after 3 attempts.")



Attempt 1: Waiting for job to complete...
Job completed, processing tool calls...
A-1.1 RequiredActionFunctionToolCall(id='call_i98nQgoQ5a0Z8Z1bOIfLEqjr', function=Function(arguments='{\n  "github_id": "all"\n}', name='get_github_data'), type='function')
Process completed.


In [61]:
# run = client.beta.threads.runs.retrieve(
#     thread_id=thread.id,
#     run_id=run.id
# )
# run

In [62]:
# Show the latest message!
time.sleep(1)
messages = client.beta.threads.messages.list(
    thread_id=thread.id
)
print(messages.model_dump_json(indent=2))


{
  "data": [
    {
      "id": "msg_iTl1JXWs1hLtXEpvSDEIMWC1",
      "assistant_id": "asst_pc2Su2UHd6IV7i3WwaYAhCqF",
      "completed_at": null,
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "The GitHub user with the most followers is yyx990803 (Evan You) with 98.7k followers."
          },
          "type": "text"
        }
      ],
      "created_at": 1712411518,
      "file_ids": [],
      "incomplete_at": null,
      "incomplete_details": null,
      "metadata": {},
      "object": "thread.message",
      "role": "assistant",
      "run_id": "run_qvTZL2sKjduAkz63iXiBar9v",
      "status": null,
      "thread_id": "thread_o8t2nKJzhk0bKprh0YYd4Rpv"
    }
  ],
  "object": "list",
  "first_id": "msg_iTl1JXWs1hLtXEpvSDEIMWC1",
  "last_id": "msg_iTl1JXWs1hLtXEpvSDEIMWC1",
  "has_more": false
}


SyncCursorPage[Message](data=[Message(id='msg_RN1sfxW7VNy5OXHyEa9IBC2d', assistant_id='asst_pc2Su2UHd6IV7i3WwaYAhCqF', completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='You have 54 repositories on GitHub.'), type='text')], created_at=1712409703, file_ids=[], incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_mAgz7AW0qiRWP6JgA2O4K1JM', status=None, thread_id='thread_sM5HtTuombiXsUFq6cpodrCE')], object='list', first_id='msg_RN1sfxW7VNy5OXHyEa9IBC2d', last_id='msg_RN1sfxW7VNy5OXHyEa9IBC2d', has_more=False)
